In [27]:
from neo4j import GraphDatabase
from neo4j import Query
import pandas as pd
import numpy as np

In [4]:
file_name = "LOTR"

In [6]:
triples = pd.read_csv("./"+file_name + "_TRIPLES.csv")
entities = pd.read_csv("./"+file_name + "_NERentities.csv")

In [7]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234"))

In [31]:

with driver.session() as session:
  res = session.run(Query("CREATE DATABASE $name IF NOT EXISTS"), name=file_name)
  res = session.run(Query("START DATABASE $name"), name=file_name)


In [50]:
def create_entity(tx, entityName, entityType):
  result = tx.run("MERGE (a:%s { name: '%s' })" % (entityType, entityName))


In [63]:
def crateRelation(tx, entity1,type1, entity2,type2, relation):
  result = tx.run("MATCH (a:%s), (b:%s) WHERE a.name = '%s' AND b.name = '%s' MERGE (a)-[r:%s]->(b)" %(type1, type2,entity1, entity2, relation) )

In [66]:
for index, row  in triples.iterrows():
  entity1 = row[1]
  entity2 = row[3]
  relation = row[2]
  relation = relation.strip()
  relation = relation.replace(" ", "_")
  splitted_1 =  entity1.split(" ")
  splitted_2 = entity2.split(" ")
  entity1_types = entities[np.isin(entities["entity"],splitted_1)]["type"].values
  entity2_types = entities[np.isin(
      entities["entity"], splitted_2)]["type"].values
  first_type = entity1_types[0] if len(entity1_types) > 0 else "Other"
  if entity2 == "farewell":
    print(entity1, entity2, relation, entity2_types)
  second_type = entity2_types[0] if len(entity2_types) > 0 else "Other"
  with driver.session() as session:
    session.write_transaction(create_entity, entity1, first_type)
    session.write_transaction(create_entity, entity2, second_type)
    session.write_transaction(crateRelation, entity1, first_type, entity2, second_type, relation)
    # res = session.run(
    #     Query("MERGE (a:$entity_1_type {name : $entity1_name})"), entity_1_type=first_type, entity1_name=entity1)
    # res = session.run(
    #     Query("MERGE (b: $entity_2_type {name : $entity2_name})"), entity2_name=entity2, entity_2_type=second_type)
    # res = session.run(Query("MATCH (a:$entity_1_type  {name : $entity1_name}), (b:$entity_2_type {name : $entity2_name}) MERGE (a)-[$relation:RELATION]->(b)"),
    #                   entity1_name=entity1, entity_1_type=first_type, entity2_name=entity2, entity_2_type=second_type, relation=relation)
 


Bilbo farewell said []
Company farewell said []
